__Вариант 1. (простой):__

__* обучить сверточную нейронную сеть на датасете fashion-mnist__

__* оценить рост точности при увеличении ширины сети (больше ядер)__

__* оценить рост точности при увеличении глубины сети (больше слоев)__

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import fashion_mnist

In [2]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [3]:
print(f"Размерность train_images: {train_images.shape}")
print(f"Размерность test_images: {test_images.shape}")

flt_size = train_images.shape[1] * train_images.shape[2]

print(f"Значения train_labels: {np.unique(train_labels)}")
print(f"Значения test_labels: {np.unique(test_labels)}")

Размерность train_images: (60000, 28, 28)
Размерность test_images: (10000, 28, 28)
Значения train_labels: [0 1 2 3 4 5 6 7 8 9]
Значения test_labels: [0 1 2 3 4 5 6 7 8 9]


In [4]:
train_images = (train_images / 255) * 2 - 1
test_images = (test_images / 255) * 2 - 1

train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

In [5]:
for N in np.arange(1, 3.1, 0.5):    
    input1= tf.keras.layers.Input(shape=(28, 28, 1))

    x1 = tf.keras.layers.Conv2D(int(8*N), kernel_size=(5, 5), strides=(1, 1), activation='tanh',  padding="same")(input1)
    x2 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(x1)
    x3 = tf.keras.layers.Conv2D(int(16*N), kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid')(x2)
    x4 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x3)
    x5 = tf.keras.layers.Conv2D(int(32*N), kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid')(x4)
    x6 = tf.keras.layers.Flatten()(x5)
    x7 = tf.keras.layers.Dense(int(20*N), activation='tanh')(x6)

    out_x = tf.keras.layers.Dense(10, activation='softmax')(x7)

    model = tf.keras.models.Model(inputs = input1, outputs = out_x) 
    model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer="Adam", metrics=["accuracy"])

    hist = model.fit(train_images,
                     tf.keras.utils.to_categorical(train_labels, 10),
                     epochs=5,
                     batch_size=256,
                     validation_data=(test_images, tf.keras.utils.to_categorical(test_labels, 10)),
                     verbose=0)

    test_score = model.evaluate(test_images, tf.keras.utils.to_categorical(test_labels, 10))
    print(f"Multiplier: {N}, test loss {test_score[0]:.4f}, accuracy {test_score[1]:.2%}")

313/313 [==============================] - 1s 3ms/step - loss: 0.3903 - accuracy: 0.8658
Multiplier: 1.0, test loss 0.3903, accuracy 86.58%
313/313 [==============================] - 1s 4ms/step - loss: 0.3691 - accuracy: 0.8703
Multiplier: 1.5, test loss 0.3691, accuracy 87.03%
313/313 [==============================] - 1s 4ms/step - loss: 0.3531 - accuracy: 0.8748
Multiplier: 2.0, test loss 0.3531, accuracy 87.48%
313/313 [==============================] - 1s 4ms/step - loss: 0.3631 - accuracy: 0.8712
Multiplier: 2.5, test loss 0.3631, accuracy 87.12%
313/313 [==============================] - 1s 4ms/step - loss: 0.3577 - accuracy: 0.8720
Multiplier: 3.0, test loss 0.3577, accuracy 87.20%


Точность сети, при увеличении ее ширины растет, но не линейно, и при определенной ширине может начать уменьшаться. Сложность вычислений, при этом, растет гораздо быстрее.

In [6]:
input1= tf.keras.layers.Input(shape=(28, 28, 1))

x1 = tf.keras.layers.Conv2D(4, kernel_size=(3, 3), strides=(1, 1), activation='tanh',  padding="same")(input1)
x2 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(x1)
x3 = tf.keras.layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding="same")(x2)
x4 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x3)
x5 = tf.keras.layers.Conv2D(12, kernel_size=(2, 2), strides=(1, 1), activation='tanh', padding="valid")(x4)
x6 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x5)
x7 = tf.keras.layers.Conv2D(16, kernel_size=(2, 2), strides=(1, 1), activation='tanh', padding="same")(x6)
x8 = tf.keras.layers.Flatten()(x7)
x9 = tf.keras.layers.Dense(32, activation='tanh')(x8)

out_x = tf.keras.layers.Dense(10, activation='softmax')(x9)

model = tf.keras.models.Model(inputs = input1, outputs = out_x) 
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer="Adam", metrics=["accuracy"])

hist = model.fit(train_images,
                 tf.keras.utils.to_categorical(train_labels, 10),
                 epochs=5,
                 batch_size=256,
                 validation_data=(test_images, tf.keras.utils.to_categorical(test_labels, 10)),
                 verbose=0)

test_score = model.evaluate(test_images, tf.keras.utils.to_categorical(test_labels, 10))
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

313/313 [==============================] - 1s 3ms/step - loss: 0.4120 - accuracy: 0.8491
Test loss 0.4120, accuracy 84.91%


In [7]:
input1= tf.keras.layers.Input(shape=(28, 28, 1))

x1 = tf.keras.layers.Conv2D(4, kernel_size=(3, 3), strides=(1, 1), activation='tanh',  padding="same")(input1)
x2 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(x1)
x3 = tf.keras.layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding="same")(x2)
x4 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x3)
x5 = tf.keras.layers.Conv2D(12, kernel_size=(2, 2), strides=(1, 1), activation='tanh', padding='valid')(x4)
x6 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x5)
x7 = tf.keras.layers.Flatten()(x6)
x8 = tf.keras.layers.Dense(32, activation='tanh')(x7)

out_x = tf.keras.layers.Dense(10, activation='softmax')(x8)

model = tf.keras.models.Model(inputs = input1, outputs = out_x) 
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer="Adam", metrics=["accuracy"])

hist = model.fit(train_images,
                 tf.keras.utils.to_categorical(train_labels, 10),
                 epochs=5,
                 batch_size=256,
                 validation_data=(test_images, tf.keras.utils.to_categorical(test_labels, 10)),
                 verbose=0)

test_score = model.evaluate(test_images, tf.keras.utils.to_categorical(test_labels, 10))
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

313/313 [==============================] - 1s 3ms/step - loss: 0.4185 - accuracy: 0.8498
Test loss 0.4185, accuracy 84.98%


In [8]:
input1= tf.keras.layers.Input(shape=(28, 28, 1))

x1 = tf.keras.layers.Conv2D(4, kernel_size=(3, 3), strides=(1, 1), activation='tanh',  padding="same")(input1)
x2 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(x1)
x3 = tf.keras.layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding="same")(x2)
x4 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x3)
x5 = tf.keras.layers.Conv2D(12, kernel_size=(2, 2), strides=(1, 1), activation='tanh', padding='valid')(x4)
x6 = tf.keras.layers.Flatten()(x5)
x7 = tf.keras.layers.Dense(32, activation='tanh')(x6)

out_x = tf.keras.layers.Dense(10, activation='softmax')(x7)

model = tf.keras.models.Model(inputs = input1, outputs = out_x) 
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer="Adam", metrics=["accuracy"])

hist = model.fit(train_images,
                 tf.keras.utils.to_categorical(train_labels, 10),
                 epochs=5,
                 batch_size=256,
                 validation_data=(test_images, tf.keras.utils.to_categorical(test_labels, 10)),
                 verbose=0)

test_score = model.evaluate(test_images, tf.keras.utils.to_categorical(test_labels, 10))
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

313/313 [==============================] - 1s 3ms/step - loss: 0.3745 - accuracy: 0.8642
Test loss 0.3745, accuracy 86.42%


In [9]:
input1= tf.keras.layers.Input(shape=(28, 28, 1))

x1 = tf.keras.layers.Conv2D(4, kernel_size=(3, 3), strides=(1, 1), activation='tanh',  padding="same")(input1)
x2 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(x1)
x3 = tf.keras.layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding="same")(x2)
x4 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x3)
x5 = tf.keras.layers.Flatten()(x4)
x6 = tf.keras.layers.Dense(32, activation='tanh')(x5)

out_x = tf.keras.layers.Dense(10, activation='softmax')(x6)

model = tf.keras.models.Model(inputs = input1, outputs = out_x) 
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer="Adam", metrics=["accuracy"])

hist = model.fit(train_images,
                 tf.keras.utils.to_categorical(train_labels, 10),
                 epochs=5,
                 batch_size=256,
                 validation_data=(test_images, tf.keras.utils.to_categorical(test_labels, 10)),
                 verbose=0)

test_score = model.evaluate(test_images, tf.keras.utils.to_categorical(test_labels, 10))
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

313/313 [==============================] - 1s 3ms/step - loss: 0.3740 - accuracy: 0.8663
Test loss 0.3740, accuracy 86.63%


In [10]:
input1= tf.keras.layers.Input(shape=(28, 28, 1))

x1 = tf.keras.layers.Conv2D(4, kernel_size=(3, 3), strides=(1, 1), activation='tanh',  padding="same")(input1)
x2 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(x1)
x3 = tf.keras.layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding="same")(x2)
x4 = tf.keras.layers.Flatten()(x3)
x5 = tf.keras.layers.Dense(32, activation='tanh')(x4)

out_x = tf.keras.layers.Dense(10, activation='softmax')(x5)

model = tf.keras.models.Model(inputs = input1, outputs = out_x) 
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer="Adam", metrics=["accuracy"])

hist = model.fit(train_images,
                 tf.keras.utils.to_categorical(train_labels, 10),
                 epochs=5,
                 batch_size=256,
                 validation_data=(test_images, tf.keras.utils.to_categorical(test_labels, 10)),
                 verbose=0)

test_score = model.evaluate(test_images, tf.keras.utils.to_categorical(test_labels, 10))
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

313/313 [==============================] - 1s 3ms/step - loss: 0.3772 - accuracy: 0.8659
Test loss 0.3772, accuracy 86.59%


С уменьшением глубины сети точность нелинейно увеличивается, это вызвано большой сверткой с бОльшим количеством слоев, что сильно уменьшает выходной тензор, слишкая малая глубина уменьшит точность. Малая глубина сети покажет неплохие результаты на простых данных, но для сложных данных потребуется больше слоев обработки.

__Вариант 2. (сложный)__

__* реализовать нейронную сеть в схеме AlexNet на сifar10.__

__* оценить рост точности при увеличении ширины сети (больше ядер)__

__* оценить рост точности при увеличении глубины сети (больше слоев)__

In [11]:
from tensorflow.keras.datasets import cifar10

In [12]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

x_train = x_train.astype('float32') / 255 * 2 - 1
x_test = x_test.astype('float32') / 255 * 2 - 1

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [13]:
input1= tf.keras.layers.Input(shape=(32, 32, 3))

x1 = tf.keras.layers.Conv2D(96, kernel_size=(4, 4), strides=(1, 1), activation='tanh',  padding="same")(input1)
x2 = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding="valid")(x1)
x3 = tf.keras.layers.Conv2D(256, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding="same")(x2)
x4 = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="valid")(x3)
x5 = tf.keras.layers.Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding="same")(x4)
x6 = tf.keras.layers.Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding="same")(x5)
x7 = tf.keras.layers.Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding="same")(x6)
x8 = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="valid")(x7)
x9 = tf.keras.layers.Flatten()(x8)
x10 = tf.keras.layers.Dense(4096, activation='tanh')(x9)
x11 = tf.keras.layers.Dense(4096, activation='tanh')(x10)

out_x = tf.keras.layers.Dense(10, activation='softmax')(x11)

model = tf.keras.models.Model(inputs = input1, outputs = out_x) 
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer="Adam", metrics=["accuracy"])

hist = model.fit(x_train,
                 y_train,
                 epochs=5,
                 batch_size=256,
                 validation_data=(x_test, y_test),
                 verbose=1)

test_score = model.evaluate(x_test, y_test)
print(f"Test loss {test_score[0]:.4f}, accuracy {test_score[1]:.2%}")

Epoch 1/5
196/196 [==============================] - 104s 528ms/step - loss: 3.4260 - accuracy: 0.1069 - val_loss: 2.3937 - val_accuracy: 0.1004
Epoch 2/5
196/196 [==============================] - 101s 514ms/step - loss: 2.3665 - accuracy: 0.1082 - val_loss: 2.3465 - val_accuracy: 0.1514
Epoch 3/5
196/196 [==============================] - 102s 519ms/step - loss: 2.2223 - accuracy: 0.1786 - val_loss: 2.1552 - val_accuracy: 0.2148
Epoch 4/5
196/196 [==============================] - 102s 519ms/step - loss: 2.0143 - accuracy: 0.2268 - val_loss: 2.0671 - val_accuracy: 0.2171
Epoch 5/5
313/313 [==============================] - 9s 30ms/step - loss: 1.9201 - accuracy: 0.2450
Test loss 1.9201, accuracy 24.50%


Сеть построена на основе AlexNet. Размерность первых слоев уменьшена для входного тензора, а размерность последнего слоя - для классификации по 10 классам.
Поведение ALexNet при увеличении ширины и глубины сети аналогична варианту 1, поэтому эксперименты над ней опустим.